In [1]:
import numpy as np
import os
import time
import requests
import json
import redis
from redis.sentinel import Sentinel

In [6]:
def print_memory_info(redis_client):
    memory_info = redis_client.info('memory')
    print(f"Current Redis memory usage: {memory_info['used_memory_human']}")
    print(f"Peak memory usage: {memory_info['used_memory_peak_human']}")
    print(f"Memory limit: {memory_info['maxmemory_human']}")

def print_eviction_policy(redis_client):
    current_policy = redis_client.config_get('maxmemory-policy')
    print(f"Current policy: {current_policy}")
    
def get_evicted_cnt(redis_client):
    stats = redis_client.info()
    current_evicted = stats['evicted_keys']
    return current_evicted


In [3]:
sentinel = Sentinel([('redis-sentinel', 26379)], socket_timeout=0.1)
master_client = sentinel.master_for('mymaster', socket_timeout=0.1)    

master_info = sentinel.discover_master('mymaster')
print(f"Master: {master_info[0]}:{master_info[1]}")
print_memory_info(master_client)

print_eviction_policy(master_client)

slaves = sentinel.discover_slaves('mymaster')
for slave_host, slave_port in slaves:
    print(f"\nSlave: {slave_host}:{slave_port}")
    slave_client = redis.Redis(host=slave_host, port=slave_port, socket_timeout=0.1)    
    print_memory_info(slave_client)
    print_eviction_policy(slave_client)

Master: 172.28.0.2:6379
Current Redis memory usage: 1.68M
Peak memory usage: 1.68M
Memory limit: 3.00M
Current policy: {'maxmemory-policy': 'allkeys-lru'}

Slave: 172.28.0.4:6379
Current Redis memory usage: 1.59M
Peak memory usage: 1.59M
Memory limit: 0B
Current policy: {'maxmemory-policy': 'noeviction'}


In [8]:
for i in range(20):
    key = 'test_key_' + str(i)
    try:
        write_result = master_client.set(key, 'x' * 1024 * 100)  # 1KB of 'x' characters        
        read_result = master_client.get(key)
        print(f"Write/read {key}, evicted: {get_evicted_cnt(master_client)}")
    except Exception as e:
        print(f"Error writing {key} to Redis: {e}")


Write/read test_key_0, evicted: 60
Write/read test_key_1, evicted: 61
Write/read test_key_2, evicted: 62
Write/read test_key_3, evicted: 63
Write/read test_key_4, evicted: 64
Write/read test_key_5, evicted: 65
Write/read test_key_6, evicted: 66
Write/read test_key_7, evicted: 67
Write/read test_key_8, evicted: 68
Write/read test_key_9, evicted: 69
Write/read test_key_10, evicted: 70
Write/read test_key_11, evicted: 71
Write/read test_key_12, evicted: 72
Write/read test_key_13, evicted: 73
Write/read test_key_14, evicted: 74
Write/read test_key_15, evicted: 75
Write/read test_key_16, evicted: 76
Write/read test_key_17, evicted: 77
Write/read test_key_18, evicted: 78
Write/read test_key_19, evicted: 79


In [17]:
flag_key = "cache_stampede_flag"
flag_value = "1"
flag_expiry = 60  # seconds

set_result = master_client.set(flag_key, flag_value, ex=flag_expiry, nx=True)
if set_result:
    print("Flag successfully set on master.")

    # Optionally, use the WAIT command to ensure replication to at least 1 slave.
    # This command will block until at least 1 replica acknowledges the write or 1000 ms elapse.
    try:
        wait_result = master_client.execute_command("WAIT", 1, 1000)
        print(f"WAIT command returned: {wait_result}")
    except Exception as e:
        print(f"Error executing WAIT command: {e}")
else:
    print("Flag already exists on master; a cache stampede might be in progress.")

# Read the flag from the master to verify its state
stored_flag = master_client.get(flag_key)
if stored_flag:
    print(f"Flag read from master: {stored_flag.decode()}")
else:
    print("Flag not found on master.")

MasterNotFoundError: No master found for 'mymaster' : <redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(host=sentinel,port=26379,db=0)>)>)> - ConnectionError('Error -2 connecting to sentinel:26379. Name or service not known.')

In [ ]:
import redis
from redis.sentinel import Sentinel

def main():
    # Connect to Sentinel running on localhost:26379
    sentinel = Sentinel([('localhost', 26379)], socket_timeout=0.1)

    # Obtain a connection to the current master for the monitored master named "mymaster"
    master_client = sentinel.master_for('mymaster', socket_timeout=0.1)

    # Define the key, value, and expiry (in seconds) for the stampede flag
    flag_key = "cache_stampede_flag"
    flag_value = "1"
    flag_expiry = 60  # seconds

    # Attempt to set the flag only if it doesn't already exist (NX option)
    set_result = master_client.set(flag_key, flag_value, ex=flag_expiry, nx=True)
    if set_result:
        print("Flag successfully set on master.")

        # Optionally, use the WAIT command to ensure replication to at least 1 slave.
        # This command will block until at least 1 replica acknowledges the write or 1000 ms elapse.
        try:
            wait_result = master_client.execute_command("WAIT", 1, 1000)
            print(f"WAIT command returned: {wait_result}")
        except Exception as e:
            print(f"Error executing WAIT command: {e}")
    else:
        print("Flag already exists on master; a cache stampede might be in progress.")

    # Read the flag from the master to verify its state
    stored_flag = master_client.get(flag_key)
    if stored_flag:
        print(f"Flag read from master: {stored_flag.decode()}")
    else:
        print("Flag not found on master.")

if __name__ == '__main__':
    main()


Error writing test_key_0 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_1 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_2 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_3 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_4 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_5 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_6 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_7 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_8 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_9 to Redis: command not allowed when used memory > 'maxmemory'.


In [ ]:
result = master_client.get('test_key')
print(f"Retrieved value: {result}")


In [34]:
def get_redis_stats(redis_client):
    # 1. Get all statistics using INFO command
    all_stats = redis_client.info()
    print("all_stats:", json.dumps(all_stats, indent=2))
    
    # Get specific sections
    memory_stats = redis_client.info('memory')
    print("memory_stats:", json.dumps(memory_stats, indent=2))

    stats = redis_client.info('stats')
    print("stats:", json.dumps(stats, indent=2))

    # 2. Key statistics
    # Total number of keys
    key_count = redis_client.dbsize()
    print("key_count:", json.dumps(key_count))

    # Memory usage of specific key
    key_memory = redis_client.memory_usage('your_key')
    print("key_memory:", json.dumps(key_memory))


    # 3. Eviction statistics from INFO
    eviction_stats = {
        'evicted_keys': stats['evicted_keys'],  # Total number of keys evicted
        'used_memory': memory_stats['used_memory'],  # Total memory used in bytes
        'used_memory_human': memory_stats['used_memory_human'],  # Human readable memory
        'maxmemory': memory_stats['maxmemory'],  # Maximum memory limit
        'maxmemory_policy': memory_stats['maxmemory_policy']  # Current eviction policy
    }
    print("eviction_stats:", json.dumps(eviction_stats, indent=2))

    # 4. Memory analysis
    # Get memory stats for all keys matching a pattern
    for key in redis_client.scan_iter("pattern*"):
        size = redis_client.memory_usage(key)
        print(f"{key}: {size} bytes")

    # 5. Get hit/miss statistics
    hits_misses = {
        'keyspace_hits': stats['keyspace_hits'],     # Number of successful lookups
        'keyspace_misses': stats['keyspace_misses'], # Number of failed lookups
        'hit_rate': stats['keyspace_hits'] / (stats['keyspace_hits'] + stats['keyspace_misses'])
    }
    print("hits_misses:", json.dumps(hits_misses, indent=2))

    # 6. Get expired/evicted counts
    expired_evicted = {
        'expired_keys': stats['expired_keys'],  # Keys that expired
        'evicted_keys': stats['evicted_keys'],  # Keys that were evicted
    }
    print("expired_evicted:", json.dumps(expired_evicted, indent=2))
    

get_redis_stats(master_client)

all_stats: {
  "redis_version": "7.4.2",
  "redis_git_sha1": 0,
  "redis_git_dirty": 0,
  "redis_build_id": "58bcfb204733694a",
  "redis_mode": "standalone",
  "os": "Linux 5.15.167.4-microsoft-standard-WSL2 x86_64",
  "arch_bits": 64,
  "monotonic_clock": "POSIX clock_gettime",
  "multiplexing_api": "epoll",
  "atomicvar_api": "c11-builtin",
  "gcc_version": "12.2.0",
  "process_id": 1,
  "process_supervised": "no",
  "run_id": "26cfb107380a3a9d762aacce7456e95f1d84b897",
  "tcp_port": 6379,
  "server_time_usec": 1738522566364451,
  "uptime_in_seconds": 1465,
  "uptime_in_days": 0,
  "hz": 10,
  "configured_hz": 10,
  "lru_clock": 10469318,
  "executable": "/data/redis-server",
  "config_file": "/usr/local/etc/redis/redis.conf",
  "io_threads_active": 0,
  "listener0": {
    "name": "tcp",
    "bind": "-::*",
    "port": 6379
  },
  "connected_clients": 4,
  "cluster_connections": 0,
  "maxclients": 10000,
  "client_recent_max_input_buffer": 8,
  "client_recent_max_output_buffer": 0,
 

In [ ]:
# change in redis.conf and restart redis-master

# def set_eviction(redis_client, eviction):
#     redis_client.config_set('maxmemory-policy', eviction)
#     redis_client.flushdb()
#     print_eviction_policy(redis_client)
#     print_memory_info(redis_client)
    
# set_eviction(master_client, 'noeviction')
# set_eviction(master_client, 'allkeys-lru')
# set_eviction(master_client, 'volatile-lru')
# set_eviction(master_client, 'allkeys-lfu') 
# set_eviction(master_client, 'volatile-lfu')
# set_eviction(master_client, 'allkeys-random')
# set_eviction(master_client, 'volatile-random')
# set_eviction(master_client, 'volatile-ttl')